In [10]:
import secrets
from copy import copy

In [11]:
def generate_starting_bits(bit_len):
    '''Generates starting bits for generator of degree bit_len
       Zero sequence is unacceptable'''
    starting_bits = secrets.randbits(bit_len)
    while(starting_bits == 0):
        starting_bits =  secrets.randbits(bit_len)
    return starting_bits

In [12]:
def L1(starting_bits, bit_len):
    '''L1 generator
       Generates sequence of degree 25 and len == bit_len using polynom (x**25 ^ x**3 ^ 1)'''
    if isinstance(starting_bits,int):
        starting_bits =  '{:022b}'.format(starting_bits)
    x = copy(starting_bits)
    #print(starting_bits)
    for i in range(0, bit_len-22):
        x += str(int(x[i])^ int(x[i+1])^ int(x[i+2])^ int(x[i+3])^ int(x[i+6])^ int(x[i+7])^ int(x[i+8])^ int(x[i+11])^ int(x[i+13]) ^ int(x[i+14]) ^ int(x[i+15])^ int(x[i+17]))
    return x

In [23]:
L1_starting = generate_starting_bits(22)
bin(L1_starting)


'0b1011111010101101010000'

In [25]:
len(bin(L1_starting))

24

In [29]:
x = L1('0000000000000000000001', 100)
x
 

'0000000000000000000001000010111110000010100110100010011010101011100011100001011011010101011011111010'

In [18]:
bin(2259582202293849872165489918971)

'0b11100100001010001101010010111000101111101010000100001000100011010101011001110001110001100001111111011'

In [96]:
def L2(starting_bits, bit_len):
    '''L2 generator
       Generates sequence of degree 26 and len == bit_len using polynom (x**26 ^ x**6 ^ x**2 ^ x ^ 1)'''
    if isinstance(starting_bits,int):
        starting_bits =  '{:026b}'.format(starting_bits)
    y = copy(starting_bits)
    #print(starting_bits)
    for i in range(0, bit_len-26):
        y += str(int(y[i]) ^ int(y[i+1]) ^ int(y[i+2]) ^ int(y[i+6]))
    return y

In [97]:
L2_starting = generate_starting_bits(26)

In [98]:
y = L2(L2_starting, 200)
y

'01000000011100011010001110110111010011110010000111110101110010010110011000101110110010011110111001011110100000011000011100001110011110010101010110101101010011101111000011000100010101010011100011010010'

In [99]:
def L3(starting_bits, bit_len):
    '''L3 generator
       Generates sequence of degree 27 and len == bit_len using polynom (x**27 ^ x**5 ^ x**2 ^ x ^ 1)'''
    if isinstance(starting_bits,int):
        starting_bits =  '{:027b}'.format(starting_bits)
    s = copy(starting_bits)
    #print(starting_bits)
    for i in range(0, bit_len-27):
        s += str(int(s[i]) ^ int(s[i+1]) ^ int(s[i+2]) ^ int(s[i+5]))
    return s

In [100]:
L3_starting = generate_starting_bits(27)


In [101]:
s = L3(L3_starting, 200)
s

'01101111100001001101101111111111110000001010111110000000111010101101011111010011111110011101101110101000100110101100111011101001000100111011011010110111101000011001101011111110010010101111001101100110'

In [102]:
print('X {}, Y {}, S {}'.format(x, y, s))

X 00100100010011100010011000000011000111111000101100001101111100011110100110110001001111110101001000011100011000101110000101111111101110101111010101000001001101101010111110100100010000011110100101000011, Y 01000000011100011010001110110111010011110010000111110101110010010110011000101110110010011110111001011110100000011000011100001110011110010101010110101101010011101111000011000100010101010011100011010010, S 01101111100001001101101111111111110000001010111110000000111010101101011111010011111110011101101110101000100110101100111011101001000100111011011010110111101000011001101011111110010010101111001101100110


In [103]:
def Geffe(bit_len, L1_starting, L2_starting, L3_starting):
    '''Geffe generator. 
       First generates L1, L2 and L3 sequences of len == bit_len using given starting bits
       Then uses function z = sx ^ (1^ s)*y to generate output'''
    x = L1(L1_starting, bit_len)
    y = L2(L2_starting, bit_len)
    s = L3(L3_starting, bit_len)
    z = ''
    for i in range(bit_len):
        z+= str((int(s[i]) * int(x[i])) ^ (1 ^ int(s[i])) *int(y[i]) )
    return z

In [104]:
z = Geffe(120, L1_starting, L2_starting, L3_starting) 
z

'001001000111010100100010000000110000111110001011011101011110000111100001101111010011100101110110010111100000001111000001'

In [13]:
from scipy.stats import norm
import math
from sympy import solve, Symbol, sqrt

/home/salamander/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Alpha is constant for all n: alpha = 0.01
Beta is <= 1/(2^n). We take the biggest value beta = 1/(2**25) for l1

In [110]:
alpha = 0.01
beta = 1 / (2**25)
beta

2.9802322387695312e-08

(1-a) and (1-b) quantiles of standart normal distribution (0,1)

In [112]:
quant_a = -(norm.ppf(alpha)) 
quant_b = -(norm.ppf(beta)) 

In [113]:
quant_a

2.3263478740408408

In [114]:
quant_b

5.419983174916869

Using sympy to solve system of equations and find constants C and N

In [115]:
N = Symbol('N')
C = Symbol('C')
solve([((N/2-C) / (sqrt(N)/2)) - quant_b, ((C - N/4) / (sqrt(3*N)/4)) - quant_a], [C, N])

[(70.2525951358299, 221.096649419668)]

Constants for L1

In [116]:
C1 = 70.2525951358299
N1 = 222

By analogy for L2

In [117]:
alpha = 0.01
beta = 1 / (2**26)
beta

1.4901161193847656e-08

In [118]:
quant_a = -(norm.ppf(alpha)) 
quant_b = -(norm.ppf(beta)) 

In [119]:
N = Symbol('N')
C = Symbol('C')
solve([((N/2-C) / (sqrt(N)/2)) - quant_b, ((C - N/4) / (sqrt(3*N)/4)) - quant_a], [C, N])

[(72.3377901496099, 228.449344487362)]

In [120]:
C2 = 73
N2 = 229

Final sequence that we have to achieve as a result of Geffe generator 

In [24]:
zi = '00010101000010100010101001111111111011001101010001101110101010000001010111101001110001100111000001000011100010111101010100000101011110101010111100001100000110111011000000000111011101011011000000011110001000110011100001000110001000000000011011000101111011000001110011100011011011111110111110101100000100110101100001001101001100011000011110101010011000101000010011101001000110011011000010100000101100100000100110100011111001100100001110110001001001001011110111010011110111010101010101000111100000000001111010001000010001000100010100100101001000111010001000100100001011010010000111111001001100100001111010001001100000011101001001000110111000010110111111011010011101010010100100001111111111101110010000110000010011111100001011111001010001011011100011100010110011100101101101111100011110101101001011010110111001101010000010100110001111111100110010100110110001011011001100101100101000101010001101101100111001100101110000011000111101111111001100110110111101100101010101011100011001000010111010100010101100010111110111110001000110001110100101110111010000111110011000110010010011011111111011111111011011101101001000000000001011111010101011101010010010001010110011100101010011100101010001111000010001100111100111110111101101101111001000010101001101100001010100011100100001111000110100111000011101011101011001100001110011111010000010010111111010001110001011010100011110000011100100111001100001001111111111001001000110001000111110010011101111001010000010000111011101101011101001000110011000010011010100111100011101000100101011101001011000110010110000111100110011010011110011100101100101101100100101110101010010001010101101011101011100100100110000011010110111100001111101101110011100010101011110111010110101011101110100110011101110000010101000110110111101100010111100001011101101111010111001100110010100000110011101111111000101111110010001011011010001001011000110000000100100101011010110110010001110110011100010101111001101110010010100010001110001110000110010000101111110110011111111001100101000000001110110011010100111001100010101011111000010111000010101101111'
len(zi)

2048

Looking for keys for L1 for which R < C1

In [80]:
def L1_break(C1, N1):
    z = zi[:N1]
    res = []
    for i in range(1, 2**25-1):
        #starting_bits
        starting_bits = '{:025b}'.format(i)
        x = '{:025b}'.format(i)
        bit_len = N1
        for j in range(0, bit_len-25):
            x += str(int(x[j]) ^ int(x[j+3]))
        #print(x+'\n')
        l = []
        for d in range(N1):
            l.append(int(x[d]) ^ int(z[d]))
        R = sum(l)
        if (R < C1): 
            res.append(starting_bits)
        if (i % 100000 == 0):
            print ('i = {}, current res = {}'.format(i, res))
    return res
    

In [81]:
L1_break(C1, N1)

i = 100000, current res = []
i = 200000, current res = []
i = 300000, current res = []
i = 400000, current res = []
i = 500000, current res = []
i = 600000, current res = []
i = 700000, current res = []
i = 800000, current res = []
i = 900000, current res = []
i = 1000000, current res = []
i = 1100000, current res = []
i = 1200000, current res = []
i = 1300000, current res = []
i = 1400000, current res = ['0000101001100111000111011']
i = 1500000, current res = ['0000101001100111000111011']
i = 1600000, current res = ['0000101001100111000111011']
i = 1700000, current res = ['0000101001100111000111011']
i = 1800000, current res = ['0000101001100111000111011', '0000110100011010011110100', '0000110100011110011010100', '0000110100011110011110100']
i = 1900000, current res = ['0000101001100111000111011', '0000110100011010011110100', '0000110100011110011010100', '0000110100011110011110100', '0000111000011010011010000', '0000111000011010011010100', '0000111000011010111010100']
i = 2000000, cur

i = 3500000, current res = ['0000101001100111000111011', '0000110100011010011110100', '0000110100011110011010100', '0000110100011110011110100', '0000111000011010011010000', '0000111000011010011010100', '0000111000011010111010100', '0000111100011010111010100', '0000111100011110011010100', '0001010000011010101010100', '0001010000011010111000100', '0001010000011010111010100', '0001010000111010011010100', '0001010000111110011110100', '0001010000111110111010100', '0001010100011000111010100', '0001010100011010011010100', '0001010100011010101010100', '0001010100011010111010100', '0001010100011010111100100', '0001010100011010111110100', '0001010100011110111010100', '0001010100111110011010100', '0001010100111110111110100']
i = 3600000, current res = ['0000101001100111000111011', '0000110100011010011110100', '0000110100011110011010100', '0000110100011110011110100', '0000111000011010011010000', '0000111000011010011010100', '0000111000011010111010100', '0000111100011010111010100', '000011110001111

i = 4200000, current res = ['0000101001100111000111011', '0000110100011010011110100', '0000110100011110011010100', '0000110100011110011110100', '0000111000011010011010000', '0000111000011010011010100', '0000111000011010111010100', '0000111100011010111010100', '0000111100011110011010100', '0001010000011010101010100', '0001010000011010111000100', '0001010000011010111010100', '0001010000111010011010100', '0001010000111110011110100', '0001010000111110111010100', '0001010100011000111010100', '0001010100011010011010100', '0001010100011010101010100', '0001010100011010111010100', '0001010100011010111100100', '0001010100011010111110100', '0001010100011110111010100', '0001010100111110011010100', '0001010100111110111110100', '0001110000001010111010100', '0001110000011010011010000', '0001110000011010011010100', '0001110000011010011110100', '0001110000011010101010100', '0001110000011010111000100', '0001110000011010111010100', '0001110000011010111010110', '0001110000011010111100100', '00011100000110

i = 4600000, current res = ['0000101001100111000111011', '0000110100011010011110100', '0000110100011110011010100', '0000110100011110011110100', '0000111000011010011010000', '0000111000011010011010100', '0000111000011010111010100', '0000111100011010111010100', '0000111100011110011010100', '0001010000011010101010100', '0001010000011010111000100', '0001010000011010111010100', '0001010000111010011010100', '0001010000111110011110100', '0001010000111110111010100', '0001010100011000111010100', '0001010100011010011010100', '0001010100011010101010100', '0001010100011010111010100', '0001010100011010111100100', '0001010100011010111110100', '0001010100011110111010100', '0001010100111110011010100', '0001010100111110111110100', '0001110000001010111010100', '0001110000011010011010000', '0001110000011010011010100', '0001110000011010011110100', '0001110000011010101010100', '0001110000011010111000100', '0001110000011010111010100', '0001110000011010111010110', '0001110000011010111100100', '00011100000110

i = 5000000, current res = ['0000101001100111000111011', '0000110100011010011110100', '0000110100011110011010100', '0000110100011110011110100', '0000111000011010011010000', '0000111000011010011010100', '0000111000011010111010100', '0000111100011010111010100', '0000111100011110011010100', '0001010000011010101010100', '0001010000011010111000100', '0001010000011010111010100', '0001010000111010011010100', '0001010000111110011110100', '0001010000111110111010100', '0001010100011000111010100', '0001010100011010011010100', '0001010100011010101010100', '0001010100011010111010100', '0001010100011010111100100', '0001010100011010111110100', '0001010100011110111010100', '0001010100111110011010100', '0001010100111110111110100', '0001110000001010111010100', '0001110000011010011010000', '0001110000011010011010100', '0001110000011010011110100', '0001110000011010101010100', '0001110000011010111000100', '0001110000011010111010100', '0001110000011010111010110', '0001110000011010111100100', '00011100000110

...

i = 32800000, current res = ['0000101001100111000111011', '0000110100011010011110100', '0000110100011110011010100', '0000110100011110011110100', '0000111000011010011010000', '0000111000011010011010100', '0000111000011010111010100', '0000111100011010111010100', '0000111100011110011010100', '0001010000011010101010100', '0001010000011010111000100', '0001010000011010111010100', '0001010000111010011010100', '0001010000111110011110100', '0001010000111110111010100', '0001010100011000111010100', '0001010100011010011010100', '0001010100011010101010100', '0001010100011010111010100', '0001010100011010111100100', '0001010100011010111110100', '0001010100011110111010100', '0001010100111110011010100', '0001010100111110111110100', '0001110000001010111010100', '0001110000011010011010000', '0001110000011010011010100', '0001110000011010011110100', '0001110000011010101010100', '0001110000011010111000100', '0001110000011010111010100', '0001110000011010111010110', '0001110000011010111100100', '0001110000011

i = 33000000, current res = ['0000101001100111000111011', '0000110100011010011110100', '0000110100011110011010100', '0000110100011110011110100', '0000111000011010011010000', '0000111000011010011010100', '0000111000011010111010100', '0000111100011010111010100', '0000111100011110011010100', '0001010000011010101010100', '0001010000011010111000100', '0001010000011010111010100', '0001010000111010011010100', '0001010000111110011110100', '0001010000111110111010100', '0001010100011000111010100', '0001010100011010011010100', '0001010100011010101010100', '0001010100011010111010100', '0001010100011010111100100', '0001010100011010111110100', '0001010100011110111010100', '0001010100111110011010100', '0001010100111110111110100', '0001110000001010111010100', '0001110000011010011010000', '0001110000011010011010100', '0001110000011010011110100', '0001110000011010101010100', '0001110000011010111000100', '0001110000011010111010100', '0001110000011010111010110', '0001110000011010111100100', '0001110000011

i = 33200000, current res = ['0000101001100111000111011', '0000110100011010011110100', '0000110100011110011010100', '0000110100011110011110100', '0000111000011010011010000', '0000111000011010011010100', '0000111000011010111010100', '0000111100011010111010100', '0000111100011110011010100', '0001010000011010101010100', '0001010000011010111000100', '0001010000011010111010100', '0001010000111010011010100', '0001010000111110011110100', '0001010000111110111010100', '0001010100011000111010100', '0001010100011010011010100', '0001010100011010101010100', '0001010100011010111010100', '0001010100011010111100100', '0001010100011010111110100', '0001010100011110111010100', '0001010100111110011010100', '0001010100111110111110100', '0001110000001010111010100', '0001110000011010011010000', '0001110000011010011010100', '0001110000011010011110100', '0001110000011010101010100', '0001110000011010111000100', '0001110000011010111010100', '0001110000011010111010110', '0001110000011010111100100', '0001110000011

i = 33400000, current res = ['0000101001100111000111011', '0000110100011010011110100', '0000110100011110011010100', '0000110100011110011110100', '0000111000011010011010000', '0000111000011010011010100', '0000111000011010111010100', '0000111100011010111010100', '0000111100011110011010100', '0001010000011010101010100', '0001010000011010111000100', '0001010000011010111010100', '0001010000111010011010100', '0001010000111110011110100', '0001010000111110111010100', '0001010100011000111010100', '0001010100011010011010100', '0001010100011010101010100', '0001010100011010111010100', '0001010100011010111100100', '0001010100011010111110100', '0001010100011110111010100', '0001010100111110011010100', '0001010100111110111110100', '0001110000001010111010100', '0001110000011010011010000', '0001110000011010011010100', '0001110000011010011110100', '0001110000011010101010100', '0001110000011010111000100', '0001110000011010111010100', '0001110000011010111010110', '0001110000011010111100100', '0001110000011

['0000101001100111000111011',
 '0000110100011010011110100',
 '0000110100011110011010100',
 '0000110100011110011110100',
 '0000111000011010011010000',
 '0000111000011010011010100',
 '0000111000011010111010100',
 '0000111100011010111010100',
 '0000111100011110011010100',
 '0001010000011010101010100',
 '0001010000011010111000100',
 '0001010000011010111010100',
 '0001010000111010011010100',
 '0001010000111110011110100',
 '0001010000111110111010100',
 '0001010100011000111010100',
 '0001010100011010011010100',
 '0001010100011010101010100',
 '0001010100011010111010100',
 '0001010100011010111100100',
 '0001010100011010111110100',
 '0001010100011110111010100',
 '0001010100111110011010100',
 '0001010100111110111110100',
 '0001110000001010111010100',
 '0001110000011010011010000',
 '0001110000011010011010100',
 '0001110000011010011110100',
 '0001110000011010101010100',
 '0001110000011010111000100',
 '0001110000011010111010100',
 '0001110000011010111010110',
 '0001110000011010111100100',
 '00011100

In [ ]:
res_L1 = ['0000101001100111000111011', '0000110100011010011110100', '0000110100011110011010100', '0000110100011110011110100', '0000111000011010011010000', '0000111000011010011010100', '0000111000011010111010100', '0000111100011010111010100', '0000111100011110011010100', '0001010000011010101010100', '0001010000011010111000100', '0001010000011010111010100', '0001010000111010011010100', '0001010000111110011110100', '0001010000111110111010100', '0001010100011000111010100', '0001010100011010011010100', '0001010100011010101010100', '0001010100011010111010100', '0001010100011010111100100', '0001010100011010111110100', '0001010100011110111010100', '0001010100111110011010100', '0001010100111110111110100', '0001110000001010111010100', '0001110000011010011010000', '0001110000011010011010100', '0001110000011010011110100', '0001110000011010101010100', '0001110000011010111000100', '0001110000011010111010100', '0001110000011010111010110', '0001110000011010111100100', '0001110000011010111110100', '0001110000011110011010100', '0001110000011110111010100', '0001110000111010111010100', '0001110000111110011010000', '0001110000111110011010100', '0001110000111110011110100', '0001110000111110111010100', '0001110000111110111110100', '0001110001010010111010100', '0001110010011010111000100', '0001110100001010101010100', '0001110100001010111010110', '0001110100011010011010000', '0001110100011010011010100', '0001110100011010011110100', '0001110100011010101010100', '0001110100011010111010100', '0001110100011010111010110', '0001110100011010111110100', '0001110100011110011010000', '0001110100011110011010100', '0001110100011110011110000', '0001110100011110011110100', '0001110100011110111010100', '0001110100011110111110100', '0001110100111010011010100', '0001110100111010111010100', '0001110100111110011110100', '0001110100111110111010100', '0001110101010010111010100', '0001111000010010011010100', '0001111000011010010010100', '0001111000011010011010100', '0001111000011010011110100', '0001111000011011011010100', '0001111001011010011010100', '0001111100010010111010100', '0001111100011010011110100', '0001111100011010111010100', '0001111100011110011010100', '0001111101011010111010100', '0011000010001000101000100', '0011000010001000101000110', '0011000010001000101010100', '0011000010001100101000110', '0011000010001100101100110', '0011000010101000101000010', '0011000110001000101000110', '0011000110001000101010100', '0011000110001000101100110', '0011000110001100101000110', '0011000110001100101010100', '0011000110001100101100110', '0011000110101100101000110', '0011000110101100101100110', '0011100010001000101000110', '0011100110001000101000110', '0011100110001000101010100', '0011100110001100101100110', '0100001101000111100111111', '0100010100011010111110000', '0100010100011010111110100', '0100010100111110111110000', '0100011100011010111010100', '0100011100111110111110000', '0100011101111110111110000', '0101000100001010101010111', '0101010000011010101010110', '0101010000011010111000110', '0101010000011010111010100', '0101010000011010111110100', '0101010000011110111010100', '0101010000011110111110100', '0101010000111010111010100', '0101010000111010111110000', '0101010000111010111110100', '0101010000111110011110100', '0101010000111110111010100', '0101010100001000101010110', '0101010100001010101010110', '0101010100001010111010100', '0101010100001010111010110', '0101010100001110111110110', '0101010100011010011010000', '0101010100011010011010100', '0101010100011010011110100', '0101010100011010101010100', '0101010100011010101010110', '0101010100011010111000110', '0101010100011010111010000', '0101010100011010111010100', '0101010100011010111010110', '0101010100011010111110000', '0101010100011010111110100', '0101010100011110011110100', '0101010100011110111110100', '0101010100011110111110110', '0101010100111010011010100', '0101010100111010111010100', '0101010100111110011010100', '0101010100111110011110100', '0101010100111110111010100', '0101010100111110111110000', '0101010100111110111110100', '0101010101010010111010100', '0101011100000010111011111', '0101011100010010111010100', '0101011100011010111010100', '0101011100011010111110100', '0101011101011010111010100', '0101011101111110111110000', '0101110000011010111010100', '0101110000011010111110100', '0101110100011010111010100', '0101110100011010111110100', '0101110100011110111110100', '0110100110001100001100110', '0111000110001000101000110', '0111000110001000101010110', '0111000110001010101010110', '0111000110001100101100110', '0111000110101100001100110', '0111100010001000001100110', '0111100010001000101000110', '0111100010001100101100110', '0111100110001000001100110', '0111100110001000101000100', '0111100110001000101000110', '0111100110001100001100110', '0111100110001100101000110', '0111100110001100101100110', '0111101010001000001100110', '1000110000011010011010100', '1000110100011010011110100', '1000110100011010111010100', '1000111000011010111010100', '1000111100011010111010100', '1001010000111110011110000', '1001010100011010011110000', '1001010100011010111110000', '1001010100011010111110100', '1001010100011110111110100', '1001010100111110011110000', '1001010100111110111110000', '1001110000011010011010000', '1001110000011010011010100', '1001110000011010011110000', '1001110000011010111010100', '1001110000011010111110000', '1001110000011010111110100', '1001110000011110011110000', '1001110000011110111110100', '1001110000111110011010000', '1001110000111110011010100', '1001110000111110011110000', '1001110000111110111110100', '1001110100011010011110000', '1001110100011010011110100', '1001110100011010111010000', '1001110100011010111010100', '1001110100011010111110000', '1001110100011010111110100', '1001110100011110011010000', '1001110100011110011010100', '1001110100011110011110000', '1001110100011110011110100', '1001110100011110111110100', '1001110100111010011110000', '1001110100111110011110000', '1001110100111110111010000', '1001111000011010011010000', '1001111000011010011010100', '1011000010001100101100110', '1011000010101100101000010', '1011000110001000001100010', '1011000110001100101100110', '1011000110001100101110100', '1011000110101100101000010', '1011000110101100101100010', '1011001110101100101000010', '1011100110001100101000010', '1011100110001100101100110', '1100001101000011100011111', '1100010100011010111010100', '1100011100011010111010100', '1100011100011010111110100', '1100011100111110111110100', '1101010000011010011110000', '1101010000011110111110100', '1101010000111010111110000', '1101010000111110011110000', '1101010000111110111010000', '1101010100001110111110110', '1101010100011010011010000', '1101010100011010011110000', '1101010100011010111010100', '1101010100011010111110000', '1101010100011010111110100', '1101010100011110011110000', '1101010100011110111110100', '1101010100011110111110110', '1101010100111010111110000', '1101010100111110011110000', '1101010100111110111010000', '1101010100111110111110000', '1101010100111110111110100', '1101010101111110111110000', '1101011101011010111110100', '1101011101111110111110100', '1101110100011010011110000', '1101110100011110111110100', '1101110100111110011110000', '1111000110001100101100110', '1111000110001100101110110', '1111100010001000001100010', '1111100010001100101100110', '1111100110001100001100010', '1111100110001100101100110']
len(res_L1)
print(res_L1)


Statistic R for each of the keys from L1

In [105]:
def analyze_stats(lst, N1):
    stat = {}
    z = zi[:N1]
    for l in lst:
        x = L1(l, N1)
        R = sum(int(x[d]) ^ int(z[d]) for d in range(N1))
        stat[l] = R
    return stat

In [128]:
stats_L1 = analyze_stats(res_L1, N1)
print(stats_L1)

{'0000101001100111000111011': 70, '0000110100011010011110100': 69, '0000110100011110011010100': 69, '0000110100011110011110100': 68, '0000111000011010011010000': 69, '0000111000011010011010100': 66, '0000111000011010111010100': 67, '0000111100011010111010100': 68, '0000111100011110011010100': 70, '0001010000011010101010100': 69, '0001010000011010111000100': 67, '0001010000011010111010100': 62, '0001010000111010011010100': 70, '0001010000111110011110100': 70, '0001010000111110111010100': 70, '0001010100011000111010100': 70, '0001010100011010011010100': 68, '0001010100011010101010100': 68, '0001010100011010111010100': 63, '0001010100011010111100100': 69, '0001010100011010111110100': 64, '0001010100011110111010100': 70, '0001010100111110011010100': 68, '0001010100111110111110100': 70, '0001110000001010111010100': 70, '0001110000011010011010000': 67, '0001110000011010011010100': 60, '0001110000011010011110100': 67, '0001110000011010101010100': 66, '0001110000011010111000100': 66, '00011100

Looking for keys for L2 for which R < C2

In [126]:
def L2_break(C2, N2):
    z = zi[:N2]
    res = []
    for i in range(1500000, 2**26-1):
        y = L2(i, N2)
        #print(x+'\n')
        R = sum(int(y[d]) ^ int(z[d]) for d in range (N2))
        if (R < C2): 
            res.append(y[:26])
        if (i % 500000 == 0):
            print ('i = {}, current res = {}'.format(i, res))
    return res

In [124]:
res_L2 = L2_break(C2, N2)

i = 500000, current res = []
i = 1000000, current res = []
i = 1500000, current res = []


NameError: name 'starting_bits' is not defined

In [127]:
res_L2 = L2_break(C2, N2)

i = 1500000, current res = []
i = 2000000, current res = ['00000111000010010011100111']
i = 2500000, current res = ['00000111000010010011100111']
i = 3000000, current res = ['00000111000010010011100111']
i = 3500000, current res = ['00000111000010010011100111']
i = 4000000, current res = ['00000111000010010011100111']
i = 4500000, current res = ['00000111000010010011100111']
i = 5000000, current res = ['00000111000010010011100111']
i = 5500000, current res = ['00000111000010010011100111']
i = 6000000, current res = ['00000111000010010011100111']
i = 6500000, current res = ['00000111000010010011100111']
i = 7000000, current res = ['00000111000010010011100111']
i = 7500000, current res = ['00000111000010010011100111']
i = 8000000, current res = ['00000111000010010011100111']
i = 8500000, current res = ['00000111000010010011100111']
i = 9000000, current res = ['00000111000010010011100111']
i = 9500000, current res = ['00000111000010010011100111']
i = 10000000, current res = ['000001110000